# Data collation and analysis
This file handles collecting all collected data into one consistent table.

## Data Collection
First data is collected, then any non-gaia stellar IDs (such as HD numbers) are matched 
with a corresponding Gaia star entry. 

## Analysis
After data has been collected into a common table, mass analysis can be performed

In [26]:
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default

## Log in to Gaia
Use credentials from gaia/CREDENTIALS file

In [38]:
Gaia.login(credentials_file='gaia/CREDENTIALS')
username = 'mwidmaie'

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


## Check for existence of the stellar_radii table
Upload if it does not exist.

In [30]:
stellar_radii_table_name = 'user_' + username + ".stellar_radii"

try:
    stellar_radii = Gaia.load_table(stellar_radii_table_name)
except:
    # Table doesn't exist so we're gonna upload from the local stellar_radii.vot file
    print("Table does not exist, uploading from local machine.")
    Gaia.upload_table(upload_resource="stellar_radii.vot", table_name='stellar_radii', format="VOTable")
    stellar_radii = Gaia.load_table(stellar_radii_table_name)

Retrieving table 'user_mwidmaie.stellar_radii'
500 Error 500:
esavo.tap.TAPException: Code: 404, msg: Table 'user_mwidmaie.stellar_radii' not found.
Table does not exist, uploading from local machine.
Sending file: stellar_radii.vot
Uploaded table 'stellar_radii'.
Retrieving table 'user_mwidmaie.stellar_radii'


In [42]:
# Update Metadata Tags
Gaia.update_user_table(table_name=stellar_radii_table_name, list_of_changes=[['"ID1"', 'indexed', True], ['"RAJ2000"', 'flags', 'Ra'], ['"DEJ2000"', 'flags', 'Dec'], ['recno', 'ucd', 'ucd sample']])

Retrieving table 'user_mwidmaie.stellar_radii'
Table 'user_mwidmaie.stellar_radii' updated.


## Matching HD numbers up with Gaia source ids

This was achieved by performing the following ADQL command:

```sql
SELECT recno, source_id as gaia_id, "ID1" as HD_NUMBER, "RAJ2000", "DEJ2000", "Vmag", "Rad" as StellarRadius, 
DISTANCE(
    POINT(stellar_radii.RAJ2000, stellar_radii.DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec,
ABS(Vmag - gaia.phot_g_mean_mag) as VMag_Difference
FROM user_mwidmaie.stellar_radii AS stellar_radii
JOIN gaiaedr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(stellar_radii.RAJ2000, stellar_radii.DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(Vmag - gaia.phot_g_mean_mag) < 2.
```

The results of this operation were stored in `data/hd_stellar_radii_match.vot`

## Collating with stellar mass data
Using data from `data/hd_stellar_radii_match.vot` we can pair up the Gaia source ids from `data/stellar_masses.vot` to form a new dataset including both radii and mass, 2 of the 3 data points we need to calculate stellar core temperature